# FitFusion Studio - AI Business Assistant

## Complete Implementation using Google Gemini API + Gradio

This notebook implements an AI-powered chatbot for FitFusion Studio with 4 tools:
1. **record_customer_interest** - Capture leads
2. **record_feedback** - Log customer feedback
3. **book_trial_session** - Book free trials
4. **check_class_availability** - Check class schedules

**Author**: [Your Name]  
**Date**: October 2025

## Step 1: Clone repo and Install Dependencies

First, install all required packages.

In [ ]:
!git clone https://github.com/alialakbar47/fitness_agent.git
import os
os.chdir("/content/fitness_agent")
!pip install -q -r requirements.txt

## Step 2: Import Libraries and Configure API

load the google api key defined in your colab secrets

In [55]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
from datetime import datetime
import json
from typing import Dict, List, Any
import gradio as gr
from google.colab import userdata
import google.generativeai as genai

# Fetch key from Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("⚠️ GOOGLE_API_KEY not found in Colab secrets!")

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)
print("✅ API configured successfully")

✅ API configured successfully


## Step 3: Load Business Information

In [56]:
def load_business_info():
    """Load business summary from file"""
    try:
        with open('me/business_summary.txt', 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print("⚠️  business_summary.txt not found. Using minimal info.")
        return """FitFusion Studio is an AI-powered fitness and wellness center offering
        personal training, group classes, nutrition coaching, and wellness programs."""

BUSINESS_INFO = load_business_info()
print("✅ Business information loaded")
print(f"Preview (first 200 chars): {BUSINESS_INFO[:200]}...")

✅ Business information loaded
Preview (first 200 chars): # FitFusion Studio - Your AI-Powered Fitness Journey

## About Us
FitFusion Studio is a revolutionary fitness and wellness center that combines cutting-edge AI technology with expert human coaching to...


## Step 4: Define Tool Functions

We'll implement 4 tools:
1. **record_customer_interest** - Capture lead information
2. **record_feedback** - Log customer feedback with severity
3. **book_trial_session** - Book free trial sessions
4. **check_class_availability** - Check class schedules

In [57]:
def record_customer_interest(name: str, email: str, interest: str, phone: str = "") -> str:
    """
    TOOL 1: Record customer lead information

    Args:
        name: Customer's full name
        email: Customer's email address
        interest: What the customer is interested in
        phone: Optional phone number

    Returns:
        Confirmation message
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    lead_entry = f"""
{'='*60}
LEAD CAPTURED: {timestamp}
Name: {name}
Email: {email}
Phone: {phone if phone else 'Not provided'}
Interest: {interest}
{'='*60}
"""

    # Append to leads file
    with open('data/leads/leads.txt', 'a', encoding='utf-8') as f:
        f.write(lead_entry + '\n')

    return f"✅ Thank you {name}! Your information has been recorded. Our team will contact you at {email} within 24 hours to discuss {interest}."


def record_feedback(feedback: str, customer_name: str = "Anonymous", severity: str = "medium") -> str:
    """
    TOOL 2: Record customer feedback or complaints

    Args:
        feedback: The feedback or complaint text
        customer_name: Name of customer providing feedback (optional)
        severity: Priority level - low, medium, or high

    Returns:
        Confirmation with ticket number
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ticket_number = f"FB-{datetime.now().strftime('%Y%m%d%H%M%S')}"

    feedback_entry = f"""
{'='*60}
FEEDBACK RECEIVED: {timestamp}
Ticket #: {ticket_number}
From: {customer_name}
Severity: {severity.upper()}
Feedback: {feedback}
{'='*60}
"""

    # Append to feedback file
    with open('data/feedback/feedback.txt', 'a', encoding='utf-8') as f:
        f.write(feedback_entry + '\n')

    return f"✅ Thank you for your feedback! Ticket #{ticket_number} has been created. Our team will review this {severity} priority item and respond accordingly."


def book_trial_session(name: str, email: str, preferred_date: str, session_type: str) -> str:
    """
    TOOL 3: Book a free trial session

    Args:
        name: Customer's full name
        email: Customer's email
        preferred_date: Preferred date for trial (e.g., "2025-10-20" or "next Monday")
        session_type: Type of session - personal_training, group_class, or nutrition

    Returns:
        Booking confirmation
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    booking_id = f"TRIAL-{datetime.now().strftime('%Y%m%d%H%M%S')}"

    # Map session types to friendly names
    session_map = {
        'personal_training': 'Personal Training Session',
        'group_class': 'Group Fitness Class',
        'nutrition': 'Nutrition Consultation'
    }
    session_name = session_map.get(session_type, session_type)

    booking_entry = f"""
{'='*60}
TRIAL SESSION BOOKED: {timestamp}
Booking ID: {booking_id}
Name: {name}
Email: {email}
Session Type: {session_name}
Preferred Date: {preferred_date}
Status: PENDING CONFIRMATION
{'='*60}
"""

    # Append to bookings file
    with open('data/bookings/bookings.txt', 'a', encoding='utf-8') as f:
        f.write(booking_entry + '\n')

    return f"""✅ Excellent! Your trial {session_name} has been requested.

Booking ID: {booking_id}
Preferred Date: {preferred_date}

We'll send a confirmation email to {email} within 2 hours with available time slots.
Please bring comfortable workout clothes and arrive 10 minutes early. We're excited to meet you!"""


def check_class_availability(class_type: str, preferred_day: str) -> str:
    """
    TOOL 4: Check available class times for specific class types

    Args:
        class_type: Type of class (yoga, hiit, spin, strength, pilates, dance)
        preferred_day: Day of week (monday, tuesday, etc. or "today", "tomorrow")

    Returns:
        List of available time slots
    """
    # Mock schedule data - in real app would query database
    schedule = {
        'yoga': {
            'monday': ['6:00 AM', '12:00 PM', '6:30 PM'],
            'tuesday': ['6:30 AM', '5:30 PM', '7:00 PM'],
            'wednesday': ['6:00 AM', '12:00 PM', '6:30 PM'],
            'thursday': ['6:30 AM', '5:30 PM'],
            'friday': ['6:00 AM', '12:00 PM', '5:30 PM'],
            'saturday': ['8:00 AM', '10:00 AM'],
            'sunday': ['9:00 AM', '11:00 AM']
        },
        'hiit': {
            'monday': ['6:30 AM', '5:30 PM', '7:00 PM'],
            'tuesday': ['6:00 AM', '12:00 PM', '6:00 PM'],
            'wednesday': ['6:30 AM', '5:30 PM', '7:00 PM'],
            'thursday': ['6:00 AM', '6:00 PM'],
            'friday': ['6:30 AM', '5:00 PM'],
            'saturday': ['9:00 AM'],
            'sunday': ['10:00 AM']
        },
        'spin': {
            'monday': ['6:00 AM', '5:00 PM'],
            'tuesday': ['6:30 AM', '6:30 PM'],
            'wednesday': ['6:00 AM', '5:00 PM'],
            'thursday': ['6:30 AM', '6:30 PM'],
            'friday': ['6:00 AM', '5:00 PM'],
            'saturday': ['8:30 AM', '10:30 AM'],
            'sunday': ['9:30 AM']
        },
        'strength': {
            'monday': ['6:30 AM', '12:00 PM', '6:00 PM'],
            'tuesday': ['6:00 AM', '5:30 PM'],
            'wednesday': ['6:30 AM', '12:00 PM', '6:00 PM'],
            'thursday': ['6:00 AM', '5:30 PM'],
            'friday': ['6:30 AM', '12:00 PM'],
            'saturday': ['9:00 AM'],
            'sunday': ['10:00 AM']
        },
        'pilates': {
            'monday': ['7:00 AM', '12:30 PM', '5:30 PM'],
            'tuesday': ['7:00 AM', '6:00 PM'],
            'wednesday': ['7:00 AM', '12:30 PM', '5:30 PM'],
            'thursday': ['7:00 AM', '6:00 PM'],
            'friday': ['7:00 AM', '12:30 PM'],
            'saturday': ['10:00 AM'],
            'sunday': ['11:00 AM']
        },
        'dance': {
            'tuesday': ['7:00 PM'],
            'thursday': ['7:00 PM'],
            'saturday': ['11:00 AM'],
            'sunday': ['2:00 PM']
        }
    }

    # Normalize inputs
    class_type = class_type.lower().strip()
    preferred_day = preferred_day.lower().strip()

    # Handle "today" and "tomorrow"
    if preferred_day in ['today', 'tomorrow']:
        days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
        current_day = datetime.now().weekday()  # 0=Monday
        if preferred_day == 'tomorrow':
            current_day = (current_day + 1) % 7
        preferred_day = days[current_day]

    # Check if class type exists
    if class_type not in schedule:
        available_classes = ', '.join(schedule.keys())
        return f"⚠️  Class type '{class_type}' not found. Available classes: {available_classes}"

    # Check if day has classes
    if preferred_day not in schedule[class_type]:
        return f"⚠️  No {class_type} classes scheduled for {preferred_day.capitalize()}. Try another day!"

    # Get available times
    times = schedule[class_type][preferred_day]
    times_str = '\n'.join([f"  • {time}" for time in times])

    return f"""✅ {class_type.capitalize()} classes available on {preferred_day.capitalize()}:

{times_str}

Each class is 60 minutes. Book your spot by telling me your name, email, and preferred time!"""



print("✅ Tool Functions Defined")


✅ Tool Functions Defined


## Step 5: Test Tool Functions
Test each tool individually to verify they work correctly.

In [58]:
# Test all tools individually
print("\n" + "="*60)
print("TESTING TOOL FUNCTIONS")
print("="*60)
print("🧪 Testing Tool 1: record_customer_interest")
result1 = record_customer_interest("Test User", "test@example.com", "Weight loss program", "555-0100")
print(result1)
print()

print("🧪 Testing Tool 2: record_feedback")
result2 = record_feedback("Great experience at the gym today!", "Happy Member", "low")
print(result2)
print()

print("🧪 Testing Tool 3: book_trial_session")
result3 = book_trial_session("Jane Doe", "jane@example.com", "2025-10-25", "personal_training")
print(result3)
print()

print("🧪 Testing Tool 4: check_class_availability")
result4 = check_class_availability("yoga", "monday")
print(result4)
print()

print("✅ All tool functions working correctly!")
print("📁 Check leads.txt, feedback.txt, and bookings.txt files")


TESTING TOOL FUNCTIONS
🧪 Testing Tool 1: record_customer_interest
✅ Thank you Test User! Your information has been recorded. Our team will contact you at test@example.com within 24 hours to discuss Weight loss program.

🧪 Testing Tool 2: record_feedback
✅ Thank you for your feedback! Ticket #FB-20251017124438 has been created. Our team will review this low priority item and respond accordingly.

🧪 Testing Tool 3: book_trial_session
✅ Excellent! Your trial Personal Training Session has been requested.

Booking ID: TRIAL-20251017124438
Preferred Date: 2025-10-25

We'll send a confirmation email to jane@example.com within 2 hours with available time slots.
Please bring comfortable workout clothes and arrive 10 minutes early. We're excited to meet you!

🧪 Testing Tool 4: check_class_availability
✅ Yoga classes available on Monday:

  • 6:00 AM
  • 12:00 PM
  • 6:30 PM

Each class is 60 minutes. Book your spot by telling me your name, email, and preferred time!

✅ All tool functions workin

## Step 6: Configure Gemini API Tools

Define tool declarations for Gemini Function Calling API.

In [59]:
# Define tool declarations in Gemini format
tools = [
    {
        "function_declarations": [
            {
                "name": "record_customer_interest",
                "description": "Records a potential customer's contact information and interest. Use this when someone expresses interest in joining, wants more information, or asks to be contacted.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string", "description": "The customer's full name"},
                        "email": {"type": "string", "description": "The customer's email address"},
                        "interest": {"type": "string", "description": "What the customer is interested in (e.g., 'personal training', 'weight loss', 'membership info')"},
                        "phone": {"type": "string", "description": "Optional phone number"}
                    },
                    "required": ["name", "email", "interest"]
                }
            },
            {
                "name": "record_feedback",
                "description": "Records customer feedback, complaints, or suggestions. Use this when customers provide feedback, report issues, or make complaints.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "feedback": {"type": "string", "description": "The feedback, complaint, or suggestion text"},
                        "customer_name": {"type": "string", "description": "Name of the customer providing feedback (default: Anonymous)"},
                        "severity": {"type": "string", "description": "Priority level: 'low' for positive feedback/suggestions, 'medium' for minor issues, 'high' for serious complaints", "enum": ["low", "medium", "high"]}
                    },
                    "required": ["feedback"]
                }
            },
            {
                "name": "book_trial_session",
                "description": "Books a free trial session for a potential customer. Use when someone wants to try a class, schedule a consultation, or book their first visit.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string", "description": "Customer's full name"},
                        "email": {"type": "string", "description": "Customer's email address"},
                        "preferred_date": {"type": "string", "description": "Preferred date for the trial session (e.g., '2025-10-25', 'next Monday', 'this weekend')"},
                        "session_type": {"type": "string", "description": "Type of trial session", "enum": ["personal_training", "group_class", "nutrition"]}
                    },
                    "required": ["name", "email", "preferred_date", "session_type"]
                }
            },
            {
                "name": "check_class_availability",
                "description": "Checks what time slots are available for a specific class type on a given day. Use when customers ask about class schedules or availability.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "class_type": {"type": "string", "description": "Type of class to check", "enum": ["yoga", "hiit", "spin", "strength", "pilates", "dance"]},
                        "preferred_day": {"type": "string", "description": "Day of the week (monday, tuesday, etc.) or 'today' or 'tomorrow'"}
                    },
                    "required": ["class_type", "preferred_day"]
                }
            }
        ]
    }
]

print("✅ Tool declarations configured for Gemini API")
print(f"📋 Configured {len(tools[0]['function_declarations'])} tools")
for tool in tools[0]['function_declarations']:
    print(f"   • {tool['name']}")

✅ Tool declarations configured for Gemini API
📋 Configured 4 tools
   • record_customer_interest
   • record_feedback
   • book_trial_session
   • check_class_availability


## Step 7: Create System Prompt

In [60]:
SYSTEM_PROMPT = f"""You are the friendly and knowledgeable AI assistant for FitFusion Studio, an AI-powered fitness and wellness center.

BUSINESS INFORMATION:
{BUSINESS_INFO}

YOUR ROLE:
- Answer questions about our services, pricing, classes, team, and facilities
- Help potential customers by collecting their contact information when they show interest
- Book trial sessions for people who want to try our services
- Check and share class schedules and availability
- Record feedback, complaints, and suggestions
- Be motivating, supportive, and professional in your fitness knowledge
- Stay in character as FitFusion Studio's assistant at all times

IMPORTANT GUIDELINES:

1. INFORMATION QUERIES: Use the business information provided above to answer questions accurately. Don't make up information not provided.

2. LEAD COLLECTION: When someone expresses interest (e.g., "I'm interested in joining", "tell me more", "I want to sign up"), use the record_customer_interest tool to capture their name, email, interest area, and optionally phone number.

3. BOOKING TRIALS: When someone wants to book a trial, try a class, or schedule a consultation, use the book_trial_session tool. Make sure to get their name, email, preferred date, and what type of session they want.

4. CLASS SCHEDULES: When asked about class times or availability, use the check_class_availability tool with the class type and day they're asking about.

5. FEEDBACK: When someone provides feedback, reports a problem, or makes a complaint, use the record_feedback tool. Assess severity: 'low' for positive feedback, 'medium' for minor issues, 'high' for serious problems.

6. UNKNOWN INFORMATION: If asked something not covered in the business information, acknowledge you don't know and offer to have a team member contact them (collect their info with record_customer_interest).

7. TONE: Be encouraging about fitness goals, empathetic to challenges, and professional yet friendly. Use motivating language but avoid being pushy about sales.

SERVICES SUMMARY:
- Personal Training: One-on-one coaching with AI-powered progress tracking
- Group Classes: Yoga, HIIT, Spin, Strength, Pilates, Dance
- Nutrition Coaching: Personalized meal plans with AI optimization
- Wellness Programs: Stress management, sleep optimization, recovery

MEMBERSHIP TIERS:
- Starter: $79/month (unlimited classes, AI app, open gym)
- Growth: $149/month (adds 2 PT sessions/month, nutrition consult)
- Elite: $249/month (adds 4 PT sessions/month, unlimited nutrition, recovery services)
- Free trial class for new members!

Remember: Your goal is to help people start their fitness journey with FitFusion Studio. Be helpful, informative, and use tools when appropriate!"""

print("✅ System prompt created")
print(f"📝 Prompt length: {len(SYSTEM_PROMPT)} characters")

✅ System prompt created
📝 Prompt length: 7699 characters


## Step 8: Test API Connection

In [61]:
print("🧪 Testing basic Gemini API connection...")

try:
    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content("Say 'Hello from FitFusion!' if you can hear me.")
    print("✅ API Connection successful!")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"❌ API Connection failed: {e}")
    print("Please check your GOOGLE_API_KEY in .env file")

🧪 Testing basic Gemini API connection...
✅ API Connection successful!
Response: Hello from FitFusion!



## Step 9: Tool Execution Function

In [62]:
def execute_tool(tool_call):
    """
    Execute a tool function call from Gemini

    Args:
        tool_call: FunctionCall object from Gemini response

    Returns:
        Tool execution result
    """
    function_name = tool_call.name
    function_args = tool_call.args

    print(f"🔧 Executing tool: {function_name}")
    print(f"📥 Arguments: {function_args}")

    try:
        # Route to appropriate function
        if function_name == "record_customer_interest":
            result = record_customer_interest(
                name=function_args.get("name"),
                email=function_args.get("email"),
                interest=function_args.get("interest"),
                phone=function_args.get("phone", "")
            )
        elif function_name == "record_feedback":
            result = record_feedback(
                feedback=function_args.get("feedback"),
                customer_name=function_args.get("customer_name", "Anonymous"),
                severity=function_args.get("severity", "medium")
            )
        elif function_name == "book_trial_session":
            result = book_trial_session(
                name=function_args.get("name"),
                email=function_args.get("email"),
                preferred_date=function_args.get("preferred_date"),
                session_type=function_args.get("session_type")
            )
        elif function_name == "check_class_availability":
            result = check_class_availability(
                class_type=function_args.get("class_type"),
                preferred_day=function_args.get("preferred_day")
            )
        else:
            result = f"Error: Unknown function {function_name}"

        print(f"✅ Tool executed successfully")
        return result

    except Exception as e:
        error_msg = f"Error executing {function_name}: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

print("✅ Tool execution function ready")

✅ Tool execution function ready


## Step 10: Create FitFusion Agent Class

In [63]:
class FitFusionAgent:
    """Main agent class for FitFusion Studio chatbot"""

    def __init__(self, system_prompt: str, tools: List[Dict]):
        """Initialize the agent with system prompt and tools"""
        self.system_prompt = system_prompt
        self.tools = tools

        # Initialize Gemini model with tools
        self.model = genai.GenerativeModel(
            'gemini-2.0-flash',
            tools=self.tools,
            system_instruction=self.system_prompt
        )

        # Start chat session
        self.chat = self.model.start_chat(history=[])

        print("✅ FitFusion Agent initialized")

    def send_message(self, user_message: str) -> str:
        """
        Send a message to the agent and get response

        Args:
            user_message: User's input message

        Returns:
            Agent's response text
        """
        try:
            print(f"\n{'='*60}")
            print(f"👤 User: {user_message}")
            print(f"{'='*60}")

            # Send message to model
            response = self.chat.send_message(user_message)

            # Handle function calls in a loop (max 5 iterations to prevent infinite loops)
            max_iterations = 5
            iteration = 0

            while iteration < max_iterations:
                # Check if the first part of the response is a function call
                try:
                    first_part = response.candidates[0].content.parts[0]

                    # Check if this part has a function_call attribute (not just checking hasattr)
                    if hasattr(first_part, 'function_call') and first_part.function_call.name:
                        print("🔧 Model requesting tool execution...")

                        # Get function call
                        function_call = first_part.function_call

                        # Execute the tool
                        tool_result = execute_tool(function_call)

                        # Send tool result back to model
                        response = self.chat.send_message(
                            genai.protos.Content(
                                parts=[genai.protos.Part(
                                    function_response=genai.protos.FunctionResponse(
                                        name=function_call.name,
                                        response={'result': tool_result}
                                    )
                                )]
                            )
                        )
                        iteration += 1
                    else:
                        # No function call, break out of loop
                        break

                except (AttributeError, IndexError) as e:
                    # No valid function call structure, break
                    break

            # Now safely get the text response
            # Don't check hasattr(response, 'text') - just try to access it
            try:
                final_response = response.text
            except ValueError:
                # If there's still a function_call, something went wrong
                final_response = "I apologize, but I couldn't complete that request properly."

            print(f"🤖 Agent: {final_response}")

            return final_response

        except Exception as e:
            error_msg = f"Sorry, I encountered an error: {str(e)}"
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()  # This will help debug
            return error_msg

    def reset_chat(self):
        """Reset the chat history"""
        self.chat = self.model.start_chat(history=[])
        print("🔄 Chat history reset")

# Initialize the agent
agent = FitFusionAgent(SYSTEM_PROMPT, tools)
print("✅ Agent ready to chat!")

✅ FitFusion Agent initialized
✅ Agent ready to chat!


## Step 11: Test Agent

Test the complete agent with various scenarios.

In [65]:
print("\n" + "="*60)
print("TESTING AGENT WITH VARIOUS SCENARIOS")
print("="*60 + "\n")

test_queries = [
    "What services do you offer?",
    "I'm interested in booking a yoga session on Monday. My name is Sarah Johnson and my email is sarah.j@email.com",
    "What yoga classes are available on Monday?",
    "I want to book a trial session. My name is Mike Brown, email mike.brown@email.com, and I'd like to try a group class next Friday.",
    "The locker room was really clean today! Great job.",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Test {i} ---")
    response = agent.send_message(query)
    print()

    # Add small delay between tests
    import time
    time.sleep(2)

print("\n✅ Agent testing complete!")
print("📁 Check leads.txt, feedback.txt, and bookings.txt for recorded data")


TESTING AGENT WITH VARIOUS SCENARIOS


--- Test 1 ---

👤 User: What services do you offer?
🤖 Agent: We offer four core services at FitFusion Studio:

*   **Personal Training**: Tailored one-on-one sessions with certified trainers, enhanced by AI-driven insights.
*   **Group Classes**: A variety of engaging classes including Yoga, HIIT, Spin, Strength & Conditioning, Pilates, and Zumba.
*   **Nutrition Coaching**: Personalized nutrition plans crafted by registered dietitians, using our AI meal planning system.
*   **Wellness Programs**: Holistic services like stress management, sleep optimization, and recovery sessions.

Would you like more details on any of these?



--- Test 2 ---

👤 User: I'm interested in booking a yoga session on Monday. My name is Sarah Johnson and my email is sarah.j@email.com
🔧 Model requesting tool execution...
🔧 Executing tool: book_trial_session
📥 Arguments: <proto.marshal.collections.maps.MapComposite object at 0x7950125b9310>
✅ Tool executed successfully
🤖

## Step 12: Create Gradio Interface

In [67]:
def gradio_chat_function(message, history):
    """
    Wrapper function for Gradio ChatInterface

    Args:
        message: User's message
        history: Chat history (list of [user_msg, bot_msg] pairs)

    Returns:
        Bot's response
    """
    # Get response from agent
    response = agent.send_message(message)
    return response

# Create Gradio interface
demo = gr.ChatInterface(
    fn=gradio_chat_function,
    title="🏋️ FitFusion Studio AI Assistant",
    description="""
    Welcome to FitFusion Studio! I'm your AI-powered fitness assistant. I can help you with:

    • 📋 Information about our services, classes, and pricing
    • 📅 Checking class schedules and availability
    • 🎯 Booking free trial sessions
    • 💪 Collecting your information if you're interested in joining
    • 💬 Recording your feedback and suggestions

    **Try asking:** "What classes do you offer?" or "I want to book a trial session"
    """,
    examples=[
        "What services do you offer?",
        "What are your membership prices?",
        "What yoga classes are available on Monday?",
        "I'm interested in personal training",
        "I want to book a trial class",
        "Tell me about your team",
    ],
    theme=gr.themes.Soft(),
    chatbot=gr.Chatbot(height=500),
)

print("✅ Gradio interface created")

/tmp/ipython-input-576214228.py:40: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


✅ Gradio interface created


## Step 13: Launch Gradio Interface

🚀 This will launch the interactive chat interface in your browser!

In [68]:
# Launch the interface
print("\n" + "="*60)
print("🚀 LAUNCHING GRADIO INTERFACE")
print("="*60)
print("\nClick the URL below to open the chat interface in your browser")
print("Press Ctrl+C to stop the server\n")

demo.launch(
    share=True,  # Set to True to create a public link
    debug=True,
    server_name="0.0.0.0",  # Allows access from other devices on network
    server_port=7860
)


🚀 LAUNCHING GRADIO INTERFACE

Click the URL below to open the chat interface in your browser
Press Ctrl+C to stop the server

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://704eb61fa9321bc2c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



👤 User: who won the worldcup in 2002
🤖 Agent: I am sorry, I cannot provide information that is not related to FitFusion Studio.

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://704eb61fa9321bc2c0.gradio.live
